In [1]:
!pip install sentence-transformers scikit-learn pandas numpy PyPDF2 python-docx
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 86.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Resume Screening Using NLP

Import

In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import PyPDF2
import docx
import re

Functions to Read Resumes

In [3]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import PyPDF2
import docx
import re

# Load resume and job datasets
resume_df = pd.read_csv("/content/Dataset/Resume.csv")
job_df = pd.read_csv("/content/Dataset/job_descriptions.csv")

# Extract texts as lists
resumes_texts = resume_df['Resume_str'].tolist()
jobs = job_df['Job Title'].tolist()

In [4]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load model (replace with 'Resume_Screening.pt' if needed)
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

# Encode resumes and jobs in batches
batch_size = 16  # adjust if you have many resumes
resume_embeddings = model.encode(resumes_texts, batch_size=batch_size, show_progress_bar=True, convert_to_numpy=True)
job_embeddings = model.encode(jobs, batch_size=batch_size, show_progress_bar=True, convert_to_numpy=True)

# Compute similarity
similarity_matrix = cosine_similarity(resume_embeddings, job_embeddings)
print("Embedding and similarity computation done.")

Using device: cpu


Batches:   0%|          | 0/156 [00:00<?, ?it/s]

Batches:   0%|          | 0/143 [00:00<?, ?it/s]

Embedding and similarity computation done.


Embeddings & Similarity

In [5]:
# Load sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode resumes and job descriptions
resume_embeddings = model.encode(resumes_texts)
job_embeddings = model.encode(jobs)

# Compute similarity
similarity_matrix = cosine_similarity(resume_embeddings, job_embeddings)

Rank Resumes for Each Job

In [7]:
# Create a DataFrame to store results
results = []

for i, job in enumerate(jobs):
    sim_scores = similarity_matrix[:, i]
    top_idx = sim_scores.argsort()[::-1]  # descending order
    for idx in top_idx:
        results.append({
            "job_description": job,
            "resume_index": idx,  # index in the CSV
            "match_score": round(sim_scores[idx]*100, 2),
            "resume_text": resumes_texts[idx]  # optional: include text
        })

results_df = pd.DataFrame(results)

# Show top 3 resumes per job
top_matches = results_df.groupby('job_description').head(3)
top_matches

,job_description,resume_index,match_score,resume_text
0,Flutter Developer,1628,35.230000,SR. FREELANCE DESIGNER/PRODUCT DEVELO...
1,Flutter Developer,1239,31.809999,IMPLEMENTATION CONSULTANT Summa...
2,Flutter Developer,2293,31.280001,SOFTWARE ENGINEER Profile E...
2484,Django Developer,926,33.810001,SOFTWARE DEVELOPER Professi...
2485,Django Developer,2395,32.200001,SOFTWARE ENGINEERING CO-OP Summ...
2486,Django Developer,291,31.730000,EA INFORMATION TECHNOLOGY SPEC...
4968,Machine Learning,1707,35.150002,ENGINEERING MANAGER Summary...
4969,Machine Learning,2153,29.240000,"CORPORATE BANKING ASSISTANT, INTERN ..."
4970,Machine Learning,1762,25.330000,ENGINEERING AND QUALITY TECHNICIAN ...
7452,iOS Developer,1264,51.090000,DIGITAL CONSULTANT Summary ...


Bonus

In [8]:
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_entities(text):
    doc = nlp(text)
    skills = []
    experience = []
    for ent in doc.ents:
        if ent.label_ in ['ORG', 'SKILL', 'PRODUCT']:
            skills.append(ent.text)
        elif ent.label_ in ['DATE', 'TIME', 'DURATION']:
            experience.append(ent.text)
    return {"skills": list(set(skills)), "experience": list(set(experience))}

# Example
for resume_text in resumes_texts:
    entities = extract_entities(resume_text)
    print("Skills:", entities['skills'])
    print("Experience:", entities['experience'])

Skills: ['IHG', 'Keeps', 'Team', 'Jefferson College         Skills     Accounting', 'Current      Company Name', 'Human Resources', 'Reservation & Front Office', 'State       Business Administration  Marketing / Advertising         High School', 'Education      N/A  ,   ', 'CPT', 'State       Awarded American Shrubel Leadership Scholarship', 'Marketing and Advertising', 'Customer Service', 'Jefferson College', 'Marketing, Customer Service', 'State     Helps', 'Customer Loyalty and Marketing', 'Chamber of Commerce', '- Executive', 'DOT', 'State', 'Business Administration']
Experience: ['1998', 'Dec 2001   to   May 2004', 'daily', '2004', '2 months', '2012', 'Dec 2010', '2007', '15+ years', 'Dec 2001', 'Dec 2013', '1999', '2010']
Skills: ['Employee Development', 'Digital Asset Management', 'Notes', 'Operations', 'Utilized', 'State', 'Microsoft Office', 'Collaborated with Communication', 'Lotus', 'Customer Care', 'RFI', 'Relational Communication    Western Connecticut State University', '

In [10]:
from sentence_transformers import SentenceTransformer
import torch

torch.save(model._first_module().state_dict(), "Resume_Screening.pt")